In [1]:
# create dataset and model sklearn
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
def fitting_model(parameters):
    N_SAMPLES = 1000
    x, y = make_classification(n_samples=N_SAMPLES, n_features=100, n_classes=2, random_state=123)
    
    n_estimators = parameters.get('n_estimators')
    max_depth = parameters.get('max_depth')
    min_samples_split = parameters.get('min_samples_split')
    min_samples_leaf = parameters.get('min_samples_leaf')
    
    rfc = RandomForestClassifier(n_estimators=n_estimators,
                                 max_depth=max_depth,
                                 min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf)
    rfc.fit(x, y)
    y_pred_proba = rfc.predict_proba(x)
    
    y_pred = y_pred_proba.argmax(1)
    print("Accuracy: ", accuracy_score(y, y_pred))

    # evaluate
    y_onehot = OneHotEncoder().fit_transform(y.reshape(-1, 1))
    loss = np.power(y_onehot.toarray() - y_pred_proba, 2)
    mse_loss = np.mean(loss)
    sem = loss.std() / np.sqrt(N_SAMPLES)
    
    return mse_loss, sem

In [4]:
from ax.core.search_space import SearchSpace
from ax import RangeParameter, ParameterType

In [5]:
n_estimators = RangeParameter(name="n_estimators",
                               parameter_type=ParameterType.INT,
                               lower=10, 
                               upper=100, 
                               log_scale=False)

max_depth = RangeParameter(name="max_depth",
                           parameter_type=ParameterType.INT,
                           lower=1, upper=5, 
                           log_scale=False)

min_samples_split = RangeParameter(name="min_samples_split",
                                   parameter_type=ParameterType.INT,
                                   lower=2,
                                   upper=5,
                                   log_scale=False)

min_samples_leaf = RangeParameter(name="min_samples_leaf",
                                  parameter_type=ParameterType.INT,
                                  lower=1,
                                  upper=5,
                                  log_scale=False)

search_space = SearchSpace([n_estimators, max_depth, min_samples_split, min_samples_leaf])

In [6]:
from ax.modelbridge.factory import get_sobol

In [7]:
m = get_sobol(search_space)
gr = m.gen(n=10)

In [ ]:
# for params in gr.arms:
#     break

# params.parameters

# params.parameters.get('learning_rate')

In [8]:
# runner
from ax import Runner

class ModelRunner(Runner):
    def __init__(self, params):
        self.params = params

    def run(self, trial):
        print(trial)
        run_metadata = evaluate(trial.arm.parameters)
        return run_metadata

    @property
    def staging_required(self):
        return False

In [9]:
from ax.core.simple_experiment import SimpleExperiment

In [10]:
def evaluate(parameters):
    return {"objective": fitting_model(parameters)}

In [12]:
exp = SimpleExperiment(
    name="simple_experiment",
    search_space=search_space,
    evaluation_function=evaluate,
    objective_name="objective",
    minimize=True
)

In [13]:
exp.runner = ModelRunner(search_space)

In [14]:
for arm in gr.arms:
    trial = exp.new_trial().add_arm(arm)
    trial.run()

Trial(experiment_name='simple_experiment', index=0, status=TrialStatus.CANDIDATE, arm=Arm(name='0_0', parameters={'n_estimators': 90, 'max_depth': 2, 'min_samples_split': 2, 'min_samples_leaf': 2}))
Accuracy:  0.979
Trial(experiment_name='simple_experiment', index=1, status=TrialStatus.CANDIDATE, arm=Arm(name='1_0', parameters={'n_estimators': 77, 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 5}))
Accuracy:  0.99
Trial(experiment_name='simple_experiment', index=2, status=TrialStatus.CANDIDATE, arm=Arm(name='2_0', parameters={'n_estimators': 25, 'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 2}))
Accuracy:  0.974
Trial(experiment_name='simple_experiment', index=3, status=TrialStatus.CANDIDATE, arm=Arm(name='3_0', parameters={'n_estimators': 13, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 3}))
Accuracy:  0.983
Trial(experiment_name='simple_experiment', index=4, status=TrialStatus.CANDIDATE, arm=Arm(name='4_0', parameters={'n_estimators': 66, 'ma

In [25]:
exp.num_trials

10